In [1]:
import cv2
import numpy as np

# Estimación de movimiento

El siguiente algoritmo realiza el seguimiento y predicción del movimiento de la pupila en el video de demostración. Para ello, se utiliza el filtro de estimación de Kalman para predecir la posición de la pupila. En general, este filtro consta de dos etapas:
<li><b>Predicción</b></li>
    En donde se utiliza la estimación a priori y la covarianza del error asociada a la estimación a priori
    $$\hat{x}_{k|k-1} = \Phi_k x_{k-1|k-1}$$ $$P_{k|k-1} = \Phi_k P_{k-1|k-1}\Phi_k^{T} + Q_k$$
<li><b>Corrección</b></li>
    En esta etapa se realiza la actualización del residuo de medición y, por consiguiente, la ganancia del estimador de Kalman
    $$\tilde{y} = z_k - H_k \hat{x}_{k|k-1} $$ $$K_k = P_{k|k-1} H_k^{T}(H_k P_{k|k-1}H_k^{T} + R_k)^{-1} $$
    $$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k \tilde{y}_k $$ $$P_{k|k} = (I - K_kH_k)P_{k|k-1} $$

In [8]:
class KF:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4,2)
        self.kf.measurementMatrix = np.array([[1,0,0,0], [0,1,0,0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    def correct(self,xpos,ypos):
        measured = np.array([[np.float32(xpos)], [np.float32(ypos)]])
        self.kf.correct(measured)
    def predict (self):
        predicted = self.kf.predict()
        return predicted

In [9]:
# get actual eye pupil position
def getEye(frame):
    lowerB = np.array([0,0,0], dtype = 'uint8')
    upperB = np.array([18,18,18], dtype = 'uint8')
    # extract mask of darker objects
    blackM = cv2.inRange(frame, lowerB, upperB)
    # morphological filter to dilate mas
    kernel = np.ones((5, 5), np.uint8)
    blackMaskDilated = cv2.dilate(blackM, kernel)
    # Find ball blob as it is the biggest black object in the frame
    [nLabels, labels, stats, centroids] = cv2.connectedComponentsWithStats(blackMaskDilated, 8, cv2.CV_32S)
    # First biggest contour is image border always, Remove it
    stats = np.delete(stats, (0), axis = 0)    
    maxBlobIdx_i, maxBlobIdx_j = np.unravel_index(stats.argmax(), stats.shape)
    # This is our ball coords that needs to be tracked
    ballX = stats[maxBlobIdx_i, 0] + (stats[maxBlobIdx_i, 2]/2)
    ballY = stats[maxBlobIdx_i, 1] + (stats[maxBlobIdx_i, 3]/2)
    return ballX, ballY

In [10]:
kf = KF()
vid = cv2.VideoCapture('eye.mp4')
i = 0

In [11]:
while (vid.isOpened()):
    rc, frame = vid.read()
    if (rc == True):
        if(True): #(i<40):
            xp, yp = getEye(frame)
            # Draw Actual coords from segmentation
            cv2.circle(frame, (int(xp), int(yp)), 40, [255,0,0], 2, 8)
            # correcting model with measurements
            kf.correct(xp,yp)
        #predict ball position with KF model
        predXY = kf.predict()    
         # Draw Kalman Filter Predicted output
        cv2.circle(frame, (int(predXY[0]), int(predXY[1])), 40, [0,255,255], 2, 8)
        cv2.imshow('Eye tracking', frame)
        i += 1
        if (cv2.waitKey(100) & 0xFF == ord('q')): break
    else: break

vid.release()
cv2.destroyAllWindows()